In [31]:
import sys
sys.path.append('/Users/celia/GitHub/mouse_bandit/data_preprocessing_code')
sys.path.append('/Users/celia/GitHub/mouse_bandit')
import support_functions as sf
import numpy as np
import pandas as pd
import scipy as sp
import bandit_preprocessing as bp
from sklearn import preprocessing
import os
import behavior_database
import shutil
import ipywidgets as widgets
from IPython.display import display

In [32]:
record_df = pd.read_csv('/Users/celia/GitHub/mouse_bandit/celia_session_record.csv',index_col=0)
celia_session_record = True

### First time we ran this, had to add 'shay' to the old data base and add a new column for Markov
    if need to troubleshoot or add trials from older data, use creating_behavior_database notebook

In [33]:
print(record_df.shape)
record_df.tail(3)

(799, 18)


,Owner,Session ID,Mouse ID,Date,Phase,Markov,Left Reward Prob,Right Reward Prob,Block Range Min,Block Range Max,No. Trials,No. Blocks,No. Rewards,p(high Port),Decision Window Duration,Min Inter-trial-interval,Left Solenoid Duration,Right Solenoid Duration
796,celia,12202017_Dill,Dill,2017-12-20,2.0,1,0.2,0.8,NaN,NaN,775.0,15.0,494.0,0.74,2.0,1.0,40.0,40.0
797,celia,12212017_Dill,Dill,2017-12-21,2.0,1,0.8,0.2,NaN,NaN,831.0,19.0,551.0,0.78,2.0,1.0,40.0,40.0
798,celia,12222017_Dill,Dill,2017-12-22,2.0,1,0.2,0.8,NaN,NaN,863.0,19.0,548.0,0.71,2.0,1.0,35.0,35.0


## Add all days (multi mice) to celia_session_record

In [34]:
if celia_session_record == True:
    record_df = record_df.drop(record_df.index[:]) # if want to reset and start with (for new session record)
    
    MOUSE = ['Baby','Mustard','Raspberry','Sky','Wine', 'Dill']
    ROOT = '/Users/celia/Dropbox (HMS)/Celia_BehaviorFiles/'
    root_mouse={}
    for mouse in MOUSE: # make list of all mouse directories
        temp_dict = {mouse:os.path.join(ROOT,mouse)}
        root_mouse.update(temp_dict)

    date_folders = os.listdir(root_mouse[MOUSE[0]]) # assuming all mice have same dates 
    date_folders = date_folders[1:]
    count=0
    for mouse in MOUSE:
        for date in date_folders:
            curr_dir = os.path.join(root_mouse[mouse], date)
            record_df = behavior_database.add_session(curr_dir, record_df, 'celia')
            count+=1
    record_df.to_csv(path_or_buf='/Users/celia/GitHub/mouse_bandit/celia_session_record.csv')

In [35]:
count - 60 # estimate for number of sessions

822

In [36]:
record_df.shape # true number of sessions in record

(808, 18)

## Moving files around
    Use MovingFilesAround notebook for moving data outside of new structure

In [39]:
# files from Celia

root_dir = '/Users/celia/Dropbox (HMS)/Celia_BehaviorFiles/'
MOUSE = ['Baby','Mustard','Raspberry','Sky','Wine']
root_mouse={}
for mouse in MOUSE: # make list of all mouse directories
    temp_dict = {mouse:os.path.join(root_dir,mouse)}
    root_mouse.update(temp_dict)
date_folders = os.listdir(root_mouse[MOUSE[0]]) # assuming all mice have same dates 
date_folders = date_folders[1:]

dist_dir = '/Users/celia/GitHub/mouse_bandit/data/trial_data'

for mouse in MOUSE:
    for date in date_folders:
        for file in os.listdir(os.path.join(root_mouse[mouse], date)):

            if not file[0] == '.':
                file_name = os.path.join(root_mouse[mouse],date,file)
                if 'trials.csv' in file:
                    shutil.copy(file_name, dist_dir)
                    #print(file_name)
                else:
                    pass
                    #print(date_folders)
                    #print(root_mouse[mouse])
                    #raise ValueError('In a hidden folder')
                    
MOUSE = ['Dill']
root_mouse={}
for mouse in MOUSE: # make list of all mouse directories
    temp_dict = {mouse:os.path.join(root_dir,mouse)}
    root_mouse.update(temp_dict)
date_folders = os.listdir(root_mouse[MOUSE[0]]) # assuming all mice have same dates 
date_folders = date_folders[1:]

dist_dir = '/Users/celia/GitHub/mouse_bandit/data/trial_data'

for mouse in MOUSE:
    for date in date_folders:
        for file in os.listdir(os.path.join(root_mouse[mouse], date)):

            if not file[0] == '.':
                file_name = os.path.join(root_mouse[mouse],date,file)
                if 'trials.csv' in file:
                    shutil.copy(file_name, dist_dir)
                    #print(file_name)
                else:
                    pass
                    #print(date_folders)
                    #print(root_mouse[mouse])
                    #raise ValueError('In a hidden folder')
        

## Make feature matrix for each condition type

In [40]:
db = pd.read_csv('/Users/celia/GitHub/mouse_bandit/celia_session_record.csv',index_col=0)
db.shape # just to double check size updated

(808, 18)

#### all markov sessions with >0.7 performance

In [41]:
r = db[(db['Markov']==1)]
r = r[r['p(high Port)'] > 0.7].copy()
r = r[r['No. Blocks']>0.0].copy()
session_names = r['Session ID'].values

In [42]:
'''
load in trial data
'''
columns = ['Elapsed Time (s)','Since last trial (s)','Trial Duration (s)','Port Poked',
           'Right Reward Prob','Left Reward Prob','Reward Given',
          'center_frame','decision_frame', 'Block ID']

root_dir = '/Users/celia/GitHub/mouse_bandit/data/trial_data'
save_dir = '/Users/celia/GitHub/mouse_bandit/data/processed_data'

trial_df = []

for session in session_names:
    full_name = session + '_trials.csv'
    
    path_name = os.path.join(root_dir,full_name)
    
    trial_df.append(pd.read_csv(path_name,names=columns))

mouse_ids = r['Mouse ID'].values

In [43]:
progress = widgets.IntProgress(min=0, max=len(trial_df), step=1)
display(progress)
for i,df in enumerate(trial_df):
    curr_feature_matrix = bp.create_feature_matrix(df,10,mouse_ids[i],session_names[i],feature_names='Default')
    
    if i == 0:
        master_matrix = curr_feature_matrix.copy()
    else:
        master_matrix = master_matrix.append(curr_feature_matrix)
    progress.value+=1

In [44]:
master_matrix.to_csv(os.path.join(save_dir,'markov_master.csv'),index=True)

##### 70-30 sessions

In [77]:
r = db[(db['Markov']==1)]
r = r[((r['Left Reward Prob'] == 0.7) |  (db['Left Reward Prob'] == 0.3))].copy()
r = r[r['p(high Port)'] > 0.7].copy()
r = r[r['No. Blocks']>0.0].copy()
session_names = r['Session ID'].values

/Users/celia/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


In [78]:
'''
load in trial data
'''
columns = ['Elapsed Time (s)','Since last trial (s)','Trial Duration (s)','Port Poked',
           'Right Reward Prob','Left Reward Prob','Reward Given',
          'center_frame','decision_frame', 'Block ID']

root_dir = '/Users/celia/GitHub/mouse_bandit/data/trial_data'
save_dir = '/Users/celia/GitHub/mouse_bandit/data/processed_data'

trial_df = []

for session in session_names:
    full_name = session + '_trials.csv'
    
    path_name = os.path.join(root_dir,full_name)
    
    trial_df.append(pd.read_csv(path_name,names=columns))

mouse_ids = r['Mouse ID'].values

In [79]:
progress = widgets.IntProgress(min=0, max=len(trial_df), step=1)
display(progress)
for i,df in enumerate(trial_df):
    curr_feature_matrix = bp.create_feature_matrix(df,10,mouse_ids[i],session_names[i],feature_names='Default')
    
    if i == 0:
        master_matrix = curr_feature_matrix.copy()
    else:
        master_matrix = master_matrix.append(curr_feature_matrix)
    progress.value+=1

In [80]:
master_matrix.to_csv(os.path.join(save_dir,'markov_7030.csv'),index=True)

In [69]:
r[r['Session ID']=='03212018_Baby']

,Owner,Session ID,Mouse ID,Date,Phase,Markov,Left Reward Prob,Right Reward Prob,Block Range Min,Block Range Max,No. Trials,No. Blocks,No. Rewards,p(high Port),Decision Window Duration,Min Inter-trial-interval,Left Solenoid Duration,Right Solenoid Duration


In [60]:
db[db['Markov']==1]

,Owner,Session ID,Mouse ID,Date,Phase,Markov,Left Reward Prob,Right Reward Prob,Block Range Min,Block Range Max,No. Trials,No. Blocks,No. Rewards,p(high Port),Decision Window Duration,Min Inter-trial-interval,Left Solenoid Duration,Right Solenoid Duration
0,celia,01112018_Baby,Baby,2018-01-11,2.0,1,0.8,0.2,NaN,NaN,597.0,9.0,431.0,0.88,2.0,1.0,40.0,40.0
1,celia,01122018_Baby,Baby,2018-01-12,2.0,1,0.2,0.8,NaN,NaN,750.0,15.0,542.0,0.86,2.0,1.0,40.0,40.0
2,celia,01152018_Baby,Baby,2018-01-15,2.0,1,0.8,0.2,NaN,NaN,800.0,13.0,591.0,0.88,2.0,1.0,40.0,40.0
3,celia,01172018_Baby,Baby,2018-01-17,2.0,1,0.2,0.8,NaN,NaN,796.0,23.0,558.0,0.84,2.0,1.0,40.0,40.0
4,celia,01192018_Baby,Baby,2018-01-19,2.0,1,0.8,0.2,NaN,NaN,715.0,15.0,498.0,0.83,2.0,1.0,40.0,40.0
5,celia,01222018_Baby,Baby,2018-01-22,2.0,1,0.7,0.3,NaN,NaN,891.0,22.0,505.0,0.73,2.0,1.0,40.0,40.0
6,celia,01242018_Baby,Baby,2018-01-24,2.0,1,0.3,0.7,NaN,NaN,855.0,18.0,522.0,0.79,2.0,1.0,40.0,40.0
7,celia,01262018_Baby,Baby,2018-01-26,2.0,1,0.7,0.3,NaN,NaN,781.0,14.0,450.0,0.74,2.0,1.0,40.0,40.0
8,celia,01302018_Baby,Baby,2018-01-30,2.0,1,0.3,0.7,NaN,NaN,852.0,14.0,530.0,0.81,2.0,1.0,40.0,40.0
9,celia,01312018_Baby,Baby,2018-01-31,2.0,1,0.3,0.7,NaN,NaN,796.0,17.0,424.0,0.59,2.0,1.0,40.0,40.0


In [70]:
db.shape

(808, 18)

In [71]:
r.shape

(161, 18)

In [72]:
save_dir

'/Users/celia/GitHub/mouse_bandit/data/processed_data'